In [108]:
import sklearn 
import pandas as pd
import xml.etree.ElementTree as ET


__First of all they did not mentioned what sklearn version they used!!!__


As described in the paper, the first step is to select base classifiers. 
The selected base classifiers are trained with default parameter settings with 10-fold cross-validation.
As input data, the training data set and its ground truth labels, per single modality is used.
For the audio MFCC features, we set NaN values to 0, and calculate the average of each MFCC coefficient over all frames.

# Load input data


# Description:

## Available Data
There are variouse csv files and data files available. It's very messy. 
There is one file called "CoE_dataset_offical_release.zip"! 
We extract this file and use this data included their for now! 

## Meta Data
In the original paper there is no information given what is included in the metadata. 
Looking at the paper describing the data set (Right Inflight? A Dataset for Exploring the Automatic Prediction of Movies Suitable for a Watching Situation
) we found out that as metadata they used language, year published, genre, country, runtime and age rating. We assume, since the author of our paper didn't say otherwise, that they used the same metadata. 

## Visual Data: 
The visual data is provied as a csv file for each movie, containing two rows. According to the paper of the dataset they calculated following visual features, Histogram of Oriented Gradients (HOG) gray, Color Moments, local binary patterns (LBP) and Gray Level Run Length Matrix, but don't say how the csv file represents them. Also as mentioned the csv file just has two rows which would not ad up to the mentioned 4 visual features. __We are treating all values as seperate column!__

## Audio Data: 
Audio features is also provided per movie as a csv file. Each audio feature consits of 12 coefficients for multiple frames.

## Textual Data
The textual data is just one file containing the tdf-idf matrix. The first line are the row names for each word. 
While the columns are the associated movie. __There is no indication to which movie each column belongs! Thus we need to assume this!__

__For now we assume the order is the same as in the df_labled_movies dataframe!!!__



In [2]:

df_labled_movies = pd.read_csv("./data/CoE_dataset/Dev_set/dev_set_groundtruth_and_trailers.csv", sep=';')
del df_labled_movies['trailer']
df_labled_movies = df_labled_movies[['movie','filename', 'goodforairplane']]
display(df_labled_movies.head(10))


### Load Meta Data ###

def load_meta_data( filenames ): 
    
    raw_data = []
    
    for file in filenames: 
        file_path = f'./data/CoE_dataset/Dev_set/XML/{file}.xml'
        with open(file_path) as f: 
            tree = ET.parse(f)
            movie = tree.find('movie')
            
            lang = movie.get('language')
            year = movie.get('year')
            genre = movie.get('genre')
            country = movie.get('country')
            runtime = movie.get('runtime')
            age_rating = movie.get('rated')
             
            raw_data.append( (file,lang,year,genre,country,runtime,age_rating) )
    
    return pd.DataFrame(raw_data, columns=['filename','language','year','genre','country','runtime','rated'])


df_meta_data = load_meta_data( df_labled_movies['filename']  )
display(df_meta_data.head(10))
display(df_meta_data.dtypes)

### Load Visual Data ###

def load_visual_data( filenames ):
    data_list = []
    
    for file in filenames: 
        file_path = f'./data/CoE_dataset/Dev_set/vis_descriptors/{file}.csv'
        df_data = pd.read_csv(file_path,index_col=None, header=None)
        data_list.append(df_data)
        
    return pd.concat(data_list, axis = 0, keys = filenames,names=('filename','vis_data'),  sort=False)

df_visual_data = load_visual_data( df_labled_movies['filename']  )
display(df_visual_data.head(10))

### Load Audio Data ###

def load_audio_data( filenames ):
    data_list = []
    
    for file in filenames: 
        file_path = f'./data/CoE_dataset/Dev_set/audio_descriptors/{file}.csv'
        df_data = pd.read_csv(file_path,index_col=None, header=None)
        data_list.append(df_data)
        
    return pd.concat(data_list, axis = 0, keys = filenames,names=('filename','freq_coeff'),  sort=False)

df_audio_data = load_audio_data( df_labled_movies['filename']  )
display(df_audio_data.head(20))


### Load textual Data ###

def load_text_data(filenames):
    

    data_list = []
    file_path = f'./data/CoE_dataset/Dev_set/text_descriptors/tdf_idf_dev.csv'
    #somehow pandas can not really handle that the first line is row names.(at least I didn't find a better way) 
    # thus we do it a little complicated here
    header_index = pd.read_csv(file_path, index_col=0,nrows=1 ).reset_index().columns
    df_data = pd.read_csv(file_path, header=None, index_col=False,skiprows=1)
    df_data.set_index(header_index, inplace=True)
    df_data.columns = filenames
    return df_data.T #row are should be represented by movie names

df_text_data = load_text_data(df_labled_movies['filename'] )
display(df_text_data.head(20))
display(df_text_data.shape)
display(df_text_data.describe())



,movie,filename,goodforairplane
0,Seventh Son,Seventh_Son,1
1,Welcome to Me,Welcome_to_Me,0
2,The Judge,The_Judge,0
3,Transformers Age of Extinction,Transformers__Age_of_Extinction,0
4,The Normal Heart,The_Normal_Heart,1
5,The Phantom Tollbooth,The_Phantom_Tollbooth,1
6,Andaz Apna Apna,Andaz_Apna_Apna,1
7,Hotel Transylvania,Hotel_Transylvania,1
8,The Matrix,The_Matrix,1
9,Into the Wild,Into_the_Wild,1


,filename,language,year,genre,country,runtime,rated
0,Seventh_Son,English,2014,"Action, Adventure, Fantasy","USA, UK, Canada, China",102 min,PG-13
1,Welcome_to_Me,English,2014,"Comedy, Drama",USA,105 min,R
2,The_Judge,English,2014,Drama,USA,141 min,R
3,Transformers__Age_of_Extinction,English,2014,"Action, Adventure, Sci-Fi","USA, China",165 min,PG-13
4,The_Normal_Heart,English,2014,Drama,USA,132 min,TV-MA
5,The_Phantom_Tollbooth,English,1970,"Family, Adventure, Animation",USA,90 min,G
6,Andaz_Apna_Apna,Hindi,1994,"Comedy, Family, Romance",India,160 min,PG
7,Hotel_Transylvania,English,2012,"Animation, Comedy, Family",USA,91 min,PG
8,The_Matrix,English,1999,"Action, Sci-Fi","USA, Australia",136 min,R
9,Into_the_Wild,"English, Danish",2007,"Adventure, Biography, Drama",USA,148 min,R


filename    object
language    object
year        object
genre       object
country     object
runtime     object
rated       object
dtype: object

0        1        2    \
filename                        vis_data                               
Seventh_Son                     0         0.047044  0.11619  0.13633   
                                1         0.056526  0.12516  0.14628   
Welcome_to_Me                   0         0.307170  0.33422  0.33112   
                                1         0.304660  0.33193  0.33124   
The_Judge                       0         0.000000  0.00000  0.00000   
                                1         0.000000  0.00000  0.00000   
Transformers__Age_of_Extinction 0         0.199960  0.26934  0.27986   
                                1         0.189130  0.25738  0.27465   
The_Normal_Heart                0         0.000000  0.00000  0.00000   
                                1         0.000000  0.00000  0.00000   

                                               3         4         5    \
filename                        vis_data                                 
Seventh_Son                     0         0.066194  0.072554  0.172670   
                                1         0.082497  0.079331  0.175380   
Welcome_to_Me                   0         0.331240  0.311140  0.336440   
                                1         0.331380  0.307880  0.332700   
The_Judge                       0         0.000000  0.000000  0.000000   
                                1         0.000000  0.000000  0.000000   
Transformers__Age_of_Extinction 0         0.237250  0.308440  0.332420   
                                1         0.236640  0.303320  0.329890   
The_Normal_Heart                0         0.000000  0.038749  0.083701   
                                1         0.000000  0.201350  0.297900   

                                              6         7         8    \
filename                        vis_data                                
Seventh_Son                     0         0.21519  0.070574  0.071423   
                                1         0.21839  0.093521  0.074837   
Welcome_to_Me                   0         0.33616  0.344790  0.169830   
                                1         0.33357  0.343050  0.173300   
The_Judge                       0         0.00000  0.000000  0.000000   
                                1         0.00000  0.000000  0.000000   
Transformers__Age_of_Extinction 0         0.32998  0.325000  0.307350   
                                1         0.32888  0.322460  0.305430   
The_Normal_Heart                0         0.10544  0.121500  0.038749   
                                1         0.39682  0.553360  0.201350   

                                               9      ...           816  \
filename                        vis_data              ...                 
Seventh_Son                     0         0.149380    ...        731.69   
                                1         0.150250    ...        689.95   
Welcome_to_Me                   0         0.273790    ...        394.34   
                                1         0.280760    ...        397.26   
The_Judge                       0         0.000000    ...     230400.00   
                                1         0.000000    ...     230400.00   
Transformers__Age_of_Extinction 0         0.334310    ...       1112.60   
                                1         0.335510    ...       1120.60   
The_Normal_Heart                0         0.083701    ...      34463.00   
                                1         0.297900    ...      41786.00   

                                                817        818        819  \
filename                        vis_data                                    
Seventh_Son                     0            502.01   1.897000   2.278800   
                                1            474.97   2.267600   2.588700   
Welcome_to_Me                   0            167.91  20.337000  21.276000   
                                1            168.23  20.426000  21.300000   
The_Judge                       0         119950.00  

0      1      2      3      4      5      6      \
filename      freq_coeff                                                    
Seventh_Son   0             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              1             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              2             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              3             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              4             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              5             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              6             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              7             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              8             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              9             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              10            NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              11            NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              12            NaN    NaN    NaN    NaN    NaN    NaN    NaN   
Welcome_to_Me 0             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              1             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              2             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              3             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              4             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              5             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              6             NaN    NaN    NaN    NaN    NaN    NaN    NaN   

                             7         8         9      ...    23078  23079  \
filename      freq_coeff                                ...                   
Seventh_Son   0          -51.23500 -25.77500 -17.41000  ...      NaN    NaN   
              1            4.76010  10.41400  15.93500  ...      NaN    NaN   
              2           -8.65190  -6.16670  -7.37720  ...      NaN    NaN   
              3           -8.13970  -8.09110 -14.56800  ...      NaN    NaN   
              4           -1.72450   2.19680   1.14500  ...      NaN    NaN   
              5            0.93079   9.88010  15.52800  ...      NaN    NaN   
              6           -2.20740   4.47300   6.56920  ...      NaN    NaN   
              7           -2.63550  -1.67510  -6.13000  ...      NaN    NaN   
              8           -0.33020   1.28230  -0.95568  ...      NaN    NaN   
              9            0.25014   7.69770  11.97200  ...      NaN    NaN   
              10          -1.42530   6.37270  10.30700  ...      NaN    NaN   
              11          -1.03030   0.39305  -1.11160  ...      NaN    NaN   
              12           0.41208  -2.07840  -4.61110  ...      NaN    NaN   
Welcome_to_Me 0                NaN       NaN       NaN  ...      NaN    NaN   
              1                NaN       NaN       NaN  ...      NaN    NaN   
              2                NaN       NaN       NaN  ...      NaN    NaN   
              3                NaN       NaN       NaN  ...      NaN    NaN   
              4                NaN       NaN       NaN  ...      NaN    NaN   
              5                NaN       NaN       NaN  ...      NaN    NaN   
              6                NaN       NaN       NaN  ...      NaN    NaN   

                          23080  23081  23082  23083  23084  23085  23086  \
filename      freq_coeff                                                    
Seventh_Son   0             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              1             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              2             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              3             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              4             NaN    NaN    NaN    NaN    NaN    NaN    NaN   
              5             NaN    NaN    NaN    NaN 

,24000,baby,baseball,big,doc,escort,frozen,heroes,high,huck,...,years.1,york,yorks,young,young.1,younger,youngja,zebra,zellweger,zoologists
filename,,,,,,,,,,,,,,,,,,,,,
Seventh_Son,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
Welcome_to_Me,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
The_Judge,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
Transformers__Age_of_Extinction,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
The_Normal_Heart,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.051657,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
The_Phantom_Tollbooth,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
Andaz_Apna_Apna,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0
Hotel_Transylvania,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.041679,0.0,0.0,0.0,0.0
The_Matrix,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.000000,0.000000,0.0,0.159570,0.159570,0.000000,0.0,0.0,0.0,0.0


(95, 3283)

,24000,baby,baseball,big,doc,escort,frozen,heroes,high,huck,...,years.1,york,yorks,young,young.1,younger,youngja,zebra,zellweger,zoologists
count,95.0,95.000000,95.000000,95.000000,95.0,95.0,95.0,95.000000,95.000000,95.0,...,95.000000,95.000000,95.0,95.000000,95.000000,95.000000,95.0,95.000000,95.0,95.0
mean,0.0,0.002288,0.001108,0.000353,0.0,0.0,0.0,0.000753,0.001395,0.0,...,0.003679,0.002638,0.0,0.006366,0.006366,0.001488,0.0,0.000531,0.0,0.0
std,0.0,0.019976,0.010803,0.003444,0.0,0.0,0.0,0.007335,0.008181,0.0,...,0.013408,0.013584,0.0,0.021099,0.021099,0.008699,0.0,0.005178,0.0,0.0
min,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
25%,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
50%,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
75%,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0
max,0.0,0.193490,0.105290,0.033572,0.0,0.0,0.0,0.071495,0.062724,0.0,...,0.073645,0.092014,0.0,0.159570,0.159570,0.067395,0.0,0.050467,0.0,0.0


# Preprocess Data

## Description 

Beside a short description for the audio data there is no more information on how to handle the other data. For example the runtime currently is not handles as a number but as a string(object)
Since sklearn mostly expects numerical inputs, we need to encode the data. 

For different class normally you would use one-hot-encoding, but since it's not specified let's try first the easiest approach which is Labelencoding.


### Audio Data: 
As mentiones in the paper, NaN values of the audio data are set to 0 and the average of each MFCC coefficient is calculated over all frames.





In [3]:

def pre_process_audio_data():
    df_data = df_audio_data.fillna(0.0)
    return df_data.mean(axis=1)
    
def pre_process_visual_data():
    #create columns of the two rows belonging to each movie
    df_data = df_visual_data.unstack()
    return df_data
    
    
df_audio_data_processed = pre_process_audio_data()
display(df_audio_data_processed.head(20))

df_visual_data_processed = pre_process_visual_data()
display(df_visual_data_processed.head(20))

filename       freq_coeff
Seventh_Son    0             33.737346
               1             -2.259660
               2              0.822080
               3             -0.298483
               4              0.680520
               5             -0.679905
               6              0.085080
               7             -0.249879
               8             -0.025137
               9             -0.134721
               10            -0.116094
               11            -0.098648
               12             0.066234
Welcome_to_Me  0             39.561047
               1             -4.593651
               2             -0.709224
               3             -1.020713
               4              0.160524
               5              0.001964
               6             -1.487054
dtype: float64

0                   1              \
vis_data                                0         1         0         1   
filename                                                                  
Seventh_Son                      0.047044  0.056526  0.116190  0.125160   
Welcome_to_Me                    0.307170  0.304660  0.334220  0.331930   
The_Judge                        0.000000  0.000000  0.000000  0.000000   
Transformers__Age_of_Extinction  0.199960  0.189130  0.269340  0.257380   
The_Normal_Heart                 0.000000  0.000000  0.000000  0.000000   
The_Phantom_Tollbooth            0.016911  0.068953  0.014269  0.099843   
Andaz_Apna_Apna                  0.000000  0.000000  0.294160  0.294520   
Hotel_Transylvania               0.000000  0.000000  0.000000  0.000000   
The_Matrix                       0.000000  0.114860  0.000000  0.208470   
Into_the_Wild                    0.000000  0.249980  0.000000  0.322560   
Jerry_Maguire                    0.000000  0.223340  0.000000  0.278790   
Lord_of_the_Flies                0.000000  0.000000  0.000000  0.000000   
The_Great_Gatsby                 0.000000  0.000000  0.000000  0.000000   
Inherent_Vice                    0.250990  0.252360  0.338650  0.348170   
Donnie_Darko                     0.000000  0.274540  0.000000  0.283910   
Anna_Karenina                    0.000000  0.000000  0.000000  0.000000   
American_Gangster                0.231640  0.231640  0.286290  0.286290   
Full_Metal_Jacket                0.179100  0.204410  0.167630  0.163100   
Words_and_Pictures               0.267330  0.279100  0.318770  0.326630   
John_Wick                        0.256280  0.089201  0.256200  0.122960   

                                      2                   3              \
vis_data                                0         1         0         1   
filename                                                                  
Seventh_Son                      0.136330  0.146280  0.066194  0.082497   
Welcome_to_Me                    0.331120  0.331240  0.331240  0.331380   
The_Judge                        0.000000  0.000000  0.000000  0.000000   
Transformers__Age_of_Extinction  0.279860  0.274650  0.237250  0.236640   
The_Normal_Heart                 0.000000  0.000000  0.000000  0.000000   
The_Phantom_Tollbooth            0.016807  0.121550  0.031862  0.171750   
Andaz_Apna_Apna                  0.290070  0.290400  0.011351  0.011381   
Hotel_Transylvania               0.000000  0.000000  0.000000  0.000000   
The_Matrix                       0.000000  0.224650  0.000000  0.245250   
Into_the_Wild                    0.000000  0.312730  0.000000  0.243740   
Jerry_Maguire                    0.000000  0.281220  0.000000  0.253180   
Lord_of_the_Flies                0.000000  0.000000  0.000000  0.000000   
The_Great_Gatsby                 0.000000  0.000000  0.000000  0.000000   
Inherent_Vice                    0.294950  0.307040  0.183030  0.189490   
Donnie_Darko                     0.000000  0.278040  0.000000  0.300110   
Anna_Karenina                    0.000000  0.000000  0.000000  0.000000   
American_Gangster                0.300680  0.300680  0.281180  0.281180   
Full_Metal_Jacket                0.092571  0.087838  0.159180  0.143390   
Words_and_Pictures               0.308140  0.309190  0.252550  0.247950   
John_Wick                        0.235420  0.171150  0.283070  0.210950   

                                      4                ...           821  \
vis_data                                0         1    ...             0   
filename                                               ...                 
Seventh_Son                      0.072554  0.079331    ...      2.950400   
Welcome_to_Me                    0.311140  0.307880    ...     21.189000   
The_Judge                        0.000000  0.000000    ...      0.002466   
Transformers__Age_of_Extinction  0.308440  0.303320    ...     14.779000   
The_Normal_Heart                 0.038749  0.201350   

# Define Models

## Description 
These are the models described in the paper. It is not allways clear which exact models they used. (see comments)

In [4]:
from  sklearn.neighbors import KNeighborsClassifier, NearestCentroid #(not sure if this is the nearest mean classifiert) 
from  sklearn.tree import DecisionTreeClassifier
from  sklearn.linear_model import LogisticRegression
from  sklearn.svm import SVC #(not clear which SVC, there is also NuSVC )
from  sklearn.ensemble import BaggingClassifier
from  sklearn.ensemble import AdaBoostClassifier
from  sklearn.ensemble import GradientBoostingClassifier
from  sklearn.ensemble import RandomForestClassifier
from  sklearn.naive_bayes import GaussianNB # there are 3 different naive bayes classifiers, it is not stated which one they used 


model_list = [KNeighborsClassifier(),
              NearestCentroid(),
                    DecisionTreeClassifier(),
                    LogisticRegression(),
                    SVC(),
                    BaggingClassifier(),
                    AdaBoostClassifier(),
                    GradientBoostingClassifier(),
                    RandomForestClassifier(),
                    GaussianNB() 
                   ]

    


# Define Performance measures:

As mentioned in the paper the performant measueres are the following Precision and Recall and F1-Score. To be more precise the weighted average of Precision and Recall and F1-Score as stated in the dataset paper. 

In [5]:
from sklearn.model_selection import cross_validate

def calculate_metrics(clf,X,y ):
    metric =  cross_validate(clf, X, y, scoring=('precision_weighted','recall_weighted','f1_weighted'), return_train_score=False, cv=10)  
    return pd.Series({'precision':metric['test_precision_weighted'].mean(),'recall':metric['test_recall_weighted'].mean(),'F1':metric['test_f1_weighted'].mean() })

# Select Models

As defined in the paper they use 10-fold CV on the classifiers for training and keep all the classifiers where the metrics are above 0.5 for later stacking.


In [6]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np

class MultiColumnLabelEncoder:
    
    def __init__(self, columns = None):
        self.columns = columns # list of column to encode

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        
        output = X.copy()
        
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname, col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        
        return output

    def fit_transform(self, X, y=None):
        return self.fit(X, y).transform(X)
    
def getModelName( object ): 

    if hasattr(object, '__module__') and hasattr(object, '__name__'):
        return  object.__name__
    elif hasattr(object, '__module__') and hasattr(object, '__class__'):
        return  object.__class__.__name__
    else:
        raise TypeError("Could not get name of object!")
    
def evaluate_models( X, y ):
    metrics = pd.DataFrame()

    for model in model_list:

        m = calculate_metrics(model,X,y )
        metrics[getModelName(model)] = m

    return metrics.T


df_final_results = pd.DataFrame()

import warnings
warnings.filterwarnings('ignore')

In [7]:
    
df_train = pd.merge(df_labled_movies,df_meta_data, on='filename')
df_train.drop(['movie', 'filename'],axis=1, inplace=True)
display(df_train.head(2))
df_X = df_train.drop('goodforairplane',axis=1)
df_y = df_train['goodforairplane']



display("----  Lable encoded ----")
label_encoder = MultiColumnLabelEncoder(['language','year','genre','country','runtime','rated'])    
X_labelencoded = label_encoder.fit_transform(df_X)
metrics = evaluate_models(X_labelencoded, df_y)
display(metrics)

#convert runtime and year to actual number
df_X['runtime'] = df_X['runtime'].apply(lambda x: int(x.split(' ')[0]) )
df_X['year'] =  df_X['year'].apply(pd.to_numeric)

display("---- Lable encoded with float for year and runtime ----")
##optimizing encoding
label_encoder = MultiColumnLabelEncoder(['language','year','genre','country','rated'])    
X_labelencoded = label_encoder.fit_transform(df_X)
metrics = evaluate_models(X_labelencoded, df_y)
display(metrics)

display("---- Lable encoded without year ----")
label_encoder = MultiColumnLabelEncoder(['language','genre','country','rated'])    
X_labelencoded = label_encoder.fit_transform(df_X)
metrics = evaluate_models(X_labelencoded, df_y)
display(metrics)

# save the best of the for the final table 
metrics['Modality'] = 'metadata'
df_final_results = df_final_results.append(metrics)


display("---- OneHot Encoding ----")
##optimizing encoding further
X_onehotencoded = pd.get_dummies(df_X)
metrics = evaluate_models(X_onehotencoded, df_y)
display(metrics)


,goodforairplane,language,year,genre,country,runtime,rated
0,1,English,2014,"Action, Adventure, Fantasy","USA, UK, Canada, China",102 min,PG-13
1,0,English,2014,"Comedy, Drama",USA,105 min,R


'----  Lable encoded ----'

,precision,recall,F1
KNeighborsClassifier,0.519400,0.566162,0.523350
NearestCentroid,0.602922,0.583333,0.576202
DecisionTreeClassifier,0.554490,0.532929,0.510497
LogisticRegression,0.587637,0.585556,0.574014
SVC,0.297467,0.536869,0.382622
BaggingClassifier,0.559570,0.542222,0.528672
AdaBoostClassifier,0.516002,0.501717,0.490721
GradientBoostingClassifier,0.488683,0.489596,0.472525
RandomForestClassifier,0.491157,0.479394,0.472887
GaussianNB,0.474880,0.506667,0.481515


'---- Lable encoded with float for year and runtime ----'

,precision,recall,F1
KNeighborsClassifier,0.657535,0.618586,0.600698
NearestCentroid,0.466408,0.479091,0.465068
DecisionTreeClassifier,0.388442,0.399596,0.384137
LogisticRegression,0.512315,0.524040,0.493776
SVC,0.300554,0.547980,0.388116
BaggingClassifier,0.468016,0.494646,0.466295
AdaBoostClassifier,0.466827,0.474444,0.462241
GradientBoostingClassifier,0.421168,0.422929,0.407376
RandomForestClassifier,0.487867,0.467374,0.446424
GaussianNB,0.467194,0.499798,0.474040


'---- Lable encoded without year ----'

,precision,recall,F1
KNeighborsClassifier,0.632037,0.619697,0.589233
NearestCentroid,0.466408,0.479091,0.465068
DecisionTreeClassifier,0.353046,0.358283,0.342731
LogisticRegression,0.545546,0.549293,0.529509
SVC,0.300554,0.547980,0.388116
BaggingClassifier,0.484537,0.483535,0.474675
AdaBoostClassifier,0.466827,0.474444,0.462241
GradientBoostingClassifier,0.404131,0.409798,0.392225
RandomForestClassifier,0.523784,0.522828,0.504285
GaussianNB,0.517396,0.539798,0.499798


'---- OneHot Encoding ----'

,precision,recall,F1
KNeighborsClassifier,0.574384,0.554242,0.544317
NearestCentroid,0.392575,0.405354,0.387835
DecisionTreeClassifier,0.543344,0.530000,0.517561
LogisticRegression,0.335283,0.402727,0.359249
SVC,0.389456,0.453333,0.413346
BaggingClassifier,0.497511,0.498687,0.472406
AdaBoostClassifier,0.391174,0.382727,0.363925
GradientBoostingClassifier,0.340226,0.400707,0.360060
RandomForestClassifier,0.401512,0.448182,0.411144
GaussianNB,0.367316,0.400707,0.347878


In [8]:
from sklearn.preprocessing import Normalizer

################## Use textual data  ###################
display('################## Use textual data  ###################')

df_movies = df_labled_movies.drop(['movie'],axis=1)
df_train = pd.merge(df_movies,df_text_data, on='filename')
df_train.drop(['filename'],axis=1, inplace=True)
display(df_train.head(2))
df_X = df_train.drop('goodforairplane',axis=1)
df_y = df_train['goodforairplane']


display("---- RAW Data ----")
metrics = evaluate_models(df_X, df_y)
display(metrics)

# save  the final table 
metrics['Modality'] = 'textual'
df_final_results = df_final_results.append(metrics)

display("---- Normalize Data ----")
df_normalized_X = Normalizer().fit_transform(df_X)
metrics = evaluate_models(df_normalized_X, df_y)
display(metrics)




'################## Use textual data  ###################'

,goodforairplane,24000,baby,baseball,big,doc,escort,frozen,heroes,high,...,years.1,york,yorks,young,young.1,younger,youngja,zebra,zellweger,zoologists
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


'---- RAW Data ----'

,precision,recall,F1
KNeighborsClassifier,0.341994,0.465152,0.366055
NearestCentroid,0.452087,0.561111,0.461862
DecisionTreeClassifier,0.422131,0.461111,0.425476
LogisticRegression,0.300554,0.547980,0.388116
SVC,0.300554,0.547980,0.388116
BaggingClassifier,0.592572,0.574242,0.555079
AdaBoostClassifier,0.554684,0.540909,0.516097
GradientBoostingClassifier,0.596914,0.632828,0.581830
RandomForestClassifier,0.438154,0.527778,0.462173
GaussianNB,0.537073,0.558182,0.538881


'---- Normalize Data ----'

,precision,recall,F1
KNeighborsClassifier,0.506534,0.533030,0.512809
NearestCentroid,0.555174,0.555253,0.531696
DecisionTreeClassifier,0.499838,0.501515,0.467442
LogisticRegression,0.300554,0.547980,0.388116
SVC,0.300554,0.547980,0.388116
BaggingClassifier,0.488500,0.510707,0.470093
AdaBoostClassifier,0.526498,0.530606,0.508858
GradientBoostingClassifier,0.591310,0.618586,0.564127
RandomForestClassifier,0.512279,0.535960,0.498207
GaussianNB,0.537073,0.558182,0.538881


In [63]:
from sklearn.preprocessing import StandardScaler,RobustScaler

################## Use visual data  ###################
display('################## Use visual data  ###################')

df_movies = df_labled_movies.drop(['movie'],axis=1)
df_train = pd.merge(df_movies,df_visual_data_processed, on='filename')
df_train.drop(['filename'],axis=1, inplace=True)
display(df_train.head(5))
df_X = df_train.drop('goodforairplane',axis=1)
df_y = df_train['goodforairplane']


display("---- RAW Data ----")
metrics = evaluate_models(df_X, df_y)
display(metrics)

display("---- Scaled Data ----")
df_scaled_X = StandardScaler().fit_transform(df_X)
metrics = evaluate_models(df_scaled_X, df_y)
display(metrics)

# save  the final table 
metrics['Modality'] = 'visual'
df_final_results = df_final_results.append(metrics)

display("---- RobustScaler Data ----")
df_scaled_X = RobustScaler().fit_transform(df_X)
metrics = evaluate_models(df_scaled_X, df_y)
display(metrics)


'################## Use visual data  ###################'

,goodforairplane,"(0, 0)","(0, 1)","(1, 0)","(1, 1)","(2, 0)","(2, 1)","(3, 0)","(3, 1)","(4, 0)",...,"(821, 0)","(821, 1)","(822, 0)","(822, 1)","(823, 0)","(823, 1)","(824, 0)","(824, 1)","(825, 0)","(825, 1)"
0,1,0.047044,0.056526,0.11619,0.12516,0.13633,0.14628,0.066194,0.082497,0.072554,...,2.950400,3.216700,91672.0,81373.0,22207.0,21045.0,26201.0,24225.0,14542.0,13529.0
1,0,0.307170,0.304660,0.33422,0.33193,0.33112,0.33124,0.331240,0.331380,0.311140,...,21.189000,21.182000,81665.0,83171.0,13672.0,13714.0,32531.0,32774.0,13753.0,13780.0
2,0,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,...,0.002466,0.002466,729320.0,729320.0,119950.0,119950.0,230400.0,230400.0,119950.0,119950.0
3,0,0.199960,0.189130,0.26934,0.25738,0.27986,0.27465,0.237250,0.236640,0.308440,...,14.779000,14.723000,208630.0,211630.0,23968.0,24019.0,47979.0,48339.0,24059.0,24090.0
4,1,0.000000,0.000000,0.00000,0.00000,0.00000,0.00000,0.000000,0.000000,0.038749,...,7.379800,11.306000,145760.0,79962.0,20730.0,20617.0,35320.0,45216.0,20831.0,20738.0


'---- RAW Data ----'

,precision,recall,F1
KNeighborsClassifier,0.546787,0.542929,0.529648
NearestCentroid,0.405332,0.473333,0.405324
DecisionTreeClassifier,0.488338,0.511313,0.485089
LogisticRegression,0.561063,0.573939,0.555761
SVC,0.398547,0.557980,0.422500
BaggingClassifier,0.591352,0.575051,0.560527
AdaBoostClassifier,0.507047,0.509293,0.499773
GradientBoostingClassifier,0.505369,0.523535,0.507581
RandomForestClassifier,0.543468,0.544646,0.530247
GaussianNB,0.503584,0.518586,0.484453


'---- Scaled Data ----'

,precision,recall,F1
KNeighborsClassifier,0.570299,0.560000,0.538146
NearestCentroid,0.515741,0.505354,0.487731
DecisionTreeClassifier,0.499180,0.519293,0.497739
LogisticRegression,0.580084,0.549798,0.538425
SVC,0.453070,0.538889,0.448757
BaggingClassifier,0.631807,0.610202,0.595888
AdaBoostClassifier,0.507985,0.504242,0.489265
GradientBoostingClassifier,0.500521,0.519495,0.504407
RandomForestClassifier,0.515810,0.494242,0.485107
GaussianNB,0.607893,0.587273,0.573590


'---- RobustScaler Data ----'

,precision,recall,F1
KNeighborsClassifier,0.558343,0.560000,0.544837
NearestCentroid,0.576882,0.539596,0.508983
DecisionTreeClassifier,0.526818,0.531515,0.513314
LogisticRegression,0.513166,0.509394,0.501544
SVC,0.425366,0.467172,0.420111
BaggingClassifier,0.606361,0.589091,0.586925
AdaBoostClassifier,0.497985,0.493131,0.478154
GradientBoostingClassifier,0.476853,0.492222,0.477668
RandomForestClassifier,0.558086,0.550000,0.536700
GaussianNB,0.532297,0.525354,0.522063


In [62]:
from sklearn.preprocessing import StandardScaler,RobustScaler

################## Use audio data  ###################
display('################## Use audio data  ###################')

df_movies = df_labled_movies.drop(['movie'],axis=1)
df_train = pd.merge(df_movies,pd.DataFrame(df_audio_data_processed), on='filename')
df_train.drop(['filename'],axis=1, inplace=True)
display(df_train.head(5))
df_X = df_train.drop('goodforairplane',axis=1)
df_y = df_train['goodforairplane']


display("---- RAW Data ----")
metrics = evaluate_models(df_X, df_y)
display(metrics)

# save  the final table 
metrics['Modality'] = 'audio'
df_final_results = df_final_results.append(metrics)

display("---- Scaled Data ----")
df_scaled_X = StandardScaler().fit_transform(df_X)
metrics = evaluate_models(df_scaled_X, df_y)
display(metrics)

display("---- RobustScaler Data ----")
df_scaled_X = RobustScaler().fit_transform(df_X)
metrics = evaluate_models(df_scaled_X, df_y)
display(metrics)

'################## Use audio data  ###################'

,goodforairplane,0
0,1,33.737346
1,1,-2.259660
2,1,0.822080
3,1,-0.298483
4,1,0.680520


'---- RAW Data ----'

,precision,recall,F1
KNeighborsClassifier,0.506285,0.510045,0.506474
NearestCentroid,0.512770,0.543300,0.436406
DecisionTreeClassifier,0.543862,0.542441,0.541880
LogisticRegression,0.485729,0.541660,0.401713
SVC,0.544320,0.545673,0.416338
BaggingClassifier,0.537071,0.533491,0.533275
AdaBoostClassifier,0.524882,0.539982,0.486366
GradientBoostingClassifier,0.500367,0.520509,0.480814
RandomForestClassifier,0.531611,0.527852,0.528012
GaussianNB,0.523032,0.546532,0.431976


'---- Scaled Data ----'

,precision,recall,F1
KNeighborsClassifier,0.506285,0.510045,0.506474
NearestCentroid,0.512770,0.543300,0.436406
DecisionTreeClassifier,0.543862,0.542441,0.541880
LogisticRegression,0.485729,0.541660,0.401713
SVC,0.486108,0.544912,0.403628
BaggingClassifier,0.539959,0.536795,0.536662
AdaBoostClassifier,0.524882,0.539982,0.486366
GradientBoostingClassifier,0.500367,0.520509,0.480814
RandomForestClassifier,0.539537,0.535976,0.535979
GaussianNB,0.523032,0.546532,0.431976


'---- RobustScaler Data ----'

,precision,recall,F1
KNeighborsClassifier,0.506285,0.510045,0.506474
NearestCentroid,0.512770,0.543300,0.436406
DecisionTreeClassifier,0.543862,0.542441,0.541880
LogisticRegression,0.485729,0.541660,0.401713
SVC,0.505053,0.529445,0.420454
BaggingClassifier,0.527959,0.523774,0.523243
AdaBoostClassifier,0.524882,0.539982,0.486366
GradientBoostingClassifier,0.500367,0.520509,0.480814
RandomForestClassifier,0.545222,0.541628,0.541991
GaussianNB,0.523032,0.546532,0.431976


## Final base classifier filter

In [11]:
df_r = df_final_results
df_r = df_r[ (df_r['precision'] > 0.5) & (df_r['recall'] > 0.5) & (df_r['F1'] > 0.5) ]
display(df_r)


,precision,recall,F1,Modality
KNeighborsClassifier,0.632037,0.619697,0.589233,metadata
LogisticRegression,0.545546,0.549293,0.529509,metadata
RandomForestClassifier,0.523784,0.522828,0.504285,metadata
BaggingClassifier,0.592572,0.574242,0.555079,textual
AdaBoostClassifier,0.554684,0.540909,0.516097,textual
GradientBoostingClassifier,0.596914,0.632828,0.581830,textual
GaussianNB,0.537073,0.558182,0.538881,textual
KNeighborsClassifier,0.570299,0.560000,0.538146,visual
DecisionTreeClassifier,0.527395,0.540606,0.529883,visual
LogisticRegression,0.580084,0.549798,0.538425,visual


As we can see the results table looks pretty different than in the paper. There is not really enough information in the paper to be sure that we are correctly reproducing the steps. 

With the audio data there is actually not really more we could do since we just end up with one coliumn of data as descirbed in the paper, but still the metrics is not as good as in the paper. 

__Is there something wrong already when we load the data ? Wrong data?__



# Task 3.2 Feature Selection

They use LVW for feature selection as described in the mentioned paper.
What is very confusion in this section is that in the end again they refer to Table 2. as also in the previous section. 
I would conclude that in the previous section they just wanted to refer to the selected classifiers and in this section to the metrics results. 
But still we would have different classifiers. 

The implementation of the LVW shouldn't be to compilcated. Maybe there is already some code out there.

In [252]:
import random
import sys

### Implementation of LVW

Here I implemented the LVW from the pseudo code of the refered paper (since I have not found any code on it), with adapting that we use a greater F1 score, instead of a lower error.

It was not clearly stated how they actually "slightly modified" the LVW...

In [408]:
def randomSet(size):
    '''
    Returns a subset of available features
    '''  
    number_of_features = random.randint(1, size-1)
    features = np.empty(number_of_features, dtype = int)
    
    i = 0
    while i < number_of_features:
        rand = random.randint(0, size-1)
        if rand not in features:
            features[i] = rand
            i = i + 1
            
    return features  


def LearnAlgo(S1, D_X, D_Y, model):
    '''
    Calculates and returns metrics on given data frame with feature
    subset S1
    '''
    D_X = D_X[D_X.columns[S1]]
    metrics = calculate_metrics(model, D_X, D_Y)
    return metrics
    

def LVW(K, D_X, D_Y, model, output = True):
    '''
    Implementation of the Las Vegas Wrapper, according to the paper
    "Feature Selection and Classification - A probabilistic approach",
    modifed to maximizing F1 instead of minimizing error.
    '''
    k = 0
    C = 100
    metrics = {"precision": 0,
               "recall": 0,
               "F1": 0}
    
    size = D_X.columns.size
    if size == 1: # fix for dataframes with size 1
        return calculate_metrics(model, D_X, D_Y)
    
    while k < K:
        S1 = randomSet(size)
        C1 = S1.size
        metrics_1 = LearnAlgo(S1, D_X, D_Y, model)
        
        if (metrics_1["F1"] > metrics["F1"] or 
            (metrics_1["F1"] == metrics["F1"] and C1 < C)):
            if output:
                print("Current best F1 = " + str(metrics_1["F1"]) + ", size = " + str(C1))
            k = 0
            metrics = metrics_1
            C = C1
            S = S1
        
        k = k + 1

    return metrics


### Preparation of data

Here I used the code segments created in 3.1 to rebuild our needed data sets.

In [409]:
# Textual data
df_train_text = pd.merge(df_movies,df_text_data, on='filename')
df_train_text.drop(['filename'],axis=1, inplace=True)
df_X_text = df_train_text.drop('goodforairplane',axis=1)
df_y_text = df_train_text['goodforairplane']
print("text:")
display(df_X_text.head(2))


# Visual data
df_train_visual = pd.merge(df_movies,df_visual_data_processed, on='filename')
df_train_visual.drop(['filename'],axis=1, inplace=True)
print("visual:")
df_X_visual = df_train_visual.drop('goodforairplane',axis=1)
df_y_visual = df_train_visual['goodforairplane']
df_scaled_X_visual = pd.DataFrame(StandardScaler().fit_transform(df_X_visual))
display(df_scaled_X_visual.head(2))


# Audio data
df_train_audio = pd.merge(df_movies,pd.DataFrame(df_audio_data_processed), on='filename')
df_train_audio.drop(['filename'],axis=1, inplace=True)
df_X_audio = df_train_audio.drop('goodforairplane',axis=1)
df_y_audio = df_train_audio['goodforairplane']
print("audio:")
display(df_X_audio.head(2))


# Meta data
df_train_meta = pd.merge(df_labled_movies,df_meta_data, on='filename')
df_train_meta.drop(['movie', 'filename'],axis=1, inplace=True)
df_X_meta = df_train_meta.drop('goodforairplane',axis=1)
df_X_meta['runtime'] = df_X_meta['runtime'].apply(lambda x: int(x.split(' ')[0]) )
df_X_meta['year'] =  df_X_meta['year'].apply(pd.to_numeric)
df_y_meta = df_train_meta['goodforairplane']
label_encoder = MultiColumnLabelEncoder(['language','genre','country','rated'])    
X_labelencoded_meta = pd.DataFrame(label_encoder.fit_transform(df_X_meta))
print("meta:")
display(X_labelencoded_meta.head(2))

text:


,24000,baby,baseball,big,doc,escort,frozen,heroes,high,huck,...,years.1,york,yorks,young,young.1,younger,youngja,zebra,zellweger,zoologists
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


visual:


,0,1,2,3,4,5,6,7,8,9,...,1642,1643,1644,1645,1646,1647,1648,1649,1650,1651
0,-0.291687,-0.564263,0.087605,-0.292158,0.214169,-0.146948,-0.242564,-0.521025,-0.242699,-0.708481,...,-0.297358,-0.644800,-0.305564,-0.409549,-0.263803,-0.268605,-0.319259,-0.481967,-0.308609,-0.437872
1,1.837915,1.420698,1.667090,1.185060,1.608901,1.139443,1.661874,1.322064,1.485797,0.969673,...,2.276885,1.732011,-0.316270,-0.403887,-0.315008,-0.440851,-0.300164,-0.376406,-0.313341,-0.431973


audio:


,0
0,33.737346
1,-2.259660


meta:


,language,year,genre,country,runtime,rated
0,2,2014,1,29,102,5
1,2,2014,31,19,105,6


### Trys for different data modalities

In [410]:
# Audio
LVW(100, df_X_audio, df_y_audio, KNeighborsClassifier())

precision    0.506285
recall       0.510045
F1           0.506474
dtype: float64

In [411]:
# Textual
LVW(100, df_X_text, df_y_text, KNeighborsClassifier())

Current best F1 = 0.5125382678323855, size = 3124
Current best F1 = 0.5277090230031407, size = 2764
Current best F1 = 0.5462951862951863, size = 1461
Current best F1 = 0.5681105931105931, size = 1082
Current best F1 = 0.5859533712474888, size = 2733


precision    0.590361
recall       0.632828
F1           0.585953
dtype: float64

In [412]:
# Visual
LVW(100, df_scaled_X_visual, df_y_visual, KNeighborsClassifier())

Current best F1 = 0.49436711436711445, size = 1387
Current best F1 = 0.5147661597661598, size = 1602
Current best F1 = 0.5277561327561326, size = 248
Current best F1 = 0.5738172938172938, size = 1301
Current best F1 = 0.5886339586339586, size = 320
Current best F1 = 0.5997552447552448, size = 599
Current best F1 = 0.6051061901061902, size = 288
Current best F1 = 0.6085592185592186, size = 110


precision    0.618054
recall       0.611313
F1           0.608559
dtype: float64

In [434]:
# Meta
LVW(100, X_labelencoded_meta, df_y_meta, KNeighborsClassifier())

Current best F1 = 0.49977688977688983, size = 3
Current best F1 = 0.564422151922152, size = 5
Current best F1 = 0.5673261923261923, size = 4
Current best F1 = 0.5692241092241093, size = 2


precision    0.612211
recall       0.596667
F1           0.569224
dtype: float64

### Application of LVW on selected combinations

No information about seed and so on, just that in each run there are different subsets and therefore different results..

In [451]:
K = 100 # max number of runs for finding better LVW combinations -> better results, longer runtime with higher K
df_final_results_lvw = pd.DataFrame()
df_x = pd.DataFrame()
df_y = pd.DataFrame()


def str2Class(str):
    return getattr(sys.modules[__name__], str)

random.seed(123)

for index, row in df_r.iterrows():
    model = str2Class(index)
    print(getModelName(model) + " - " + row["Modality"])
    
    # get correct data frame
    if row["Modality"] == "metadata":
        df_x = X_labelencoded_meta
        df_y = df_y_meta
    elif row["Modality"] == "visual":
        df_x = df_scaled_X_visual
        df_y = df_y_visual
        
    elif row["Modality"] == "textual":
        df_x = df_X_text
        df_y = df_y_text
    else:
        df_x = df_X_audio
        df_y = df_y_audio
    
    # Calculate and append LVW metrics
    metrics = pd.DataFrame()
    
    if getModelName(model) == "RandomForestClassifier":
        print("skipping random forest..")
        # evaluate random forest withot LVW since it already performs feature selection
        m = calculate_metrics(model(), df_x, df_y)
    else:
        # feature selection metrics
        m = LVW(K, df_x, df_y, model(), False)
    print(m)
    metrics[getModelName(model)] = m
    metrics = metrics.T
    metrics["Modality"] = row["Modality"]
    
    df_final_results_lvw = df_final_results_lvw.append(metrics)


KNeighborsClassifier - metadata
precision    0.612211
recall       0.596667
F1           0.569224
dtype: float64
LogisticRegression - metadata
precision    0.580071
recall       0.605758
F1           0.562425
dtype: float64
RandomForestClassifier - metadata
skipping random forest..
precision    0.455356
recall       0.441111
F1           0.427100
dtype: float64
BaggingClassifier - textual
precision    0.663314
recall       0.658990
F1           0.623904
dtype: float64
AdaBoostClassifier - textual
precision    0.704281
recall       0.684141
F1           0.667203
dtype: float64
GradientBoostingClassifier - textual
precision    0.702160
recall       0.684343
F1           0.642548
dtype: float64
GaussianNB - textual
precision    0.718624
recall       0.674141
F1           0.663622
dtype: float64
KNeighborsClassifier - visual
precision    0.646566
recall       0.638788
F1           0.630700
dtype: float64
DecisionTreeClassifier - visual
precision    0.682261
recall       0.661717
F1        

### Final results
Here, the Random Forest for meta data is worse than before, why (TODO)?
Should we only make the feature selection results, even if its worse than the full data?

In [452]:

# Previous results
display(df_r)

# Results with feature selection
display(df_final_results_lvw)


,precision,recall,F1,Modality
KNeighborsClassifier,0.632037,0.619697,0.589233,metadata
LogisticRegression,0.545546,0.549293,0.529509,metadata
RandomForestClassifier,0.523784,0.522828,0.504285,metadata
BaggingClassifier,0.592572,0.574242,0.555079,textual
AdaBoostClassifier,0.554684,0.540909,0.516097,textual
GradientBoostingClassifier,0.596914,0.632828,0.581830,textual
GaussianNB,0.537073,0.558182,0.538881,textual
KNeighborsClassifier,0.570299,0.560000,0.538146,visual
DecisionTreeClassifier,0.527395,0.540606,0.529883,visual
LogisticRegression,0.580084,0.549798,0.538425,visual


,precision,recall,F1,Modality
KNeighborsClassifier,0.612211,0.596667,0.569224,metadata
LogisticRegression,0.580071,0.605758,0.562425,metadata
RandomForestClassifier,0.455356,0.441111,0.427100,metadata
BaggingClassifier,0.663314,0.658990,0.623904,textual
AdaBoostClassifier,0.704281,0.684141,0.667203,textual
GradientBoostingClassifier,0.702160,0.684343,0.642548,textual
GaussianNB,0.718624,0.674141,0.663622,textual
KNeighborsClassifier,0.646566,0.638788,0.630700,visual
DecisionTreeClassifier,0.682261,0.661717,0.650741,visual
LogisticRegression,0.632202,0.636667,0.623593,visual
